In [ ]:
FinalSet.append(LABEL)
rDF = pd.DataFrame(df[FinalSet]).copy()
Train70, Test30=train_test_split(rDF,test_size=0.3, random_state=10)
len(Train70.axes[0])

In [ ]:
TrainA, TrainB=train_test_split(Train70,test_size=0.5, random_state=10)
len(TrainA.axes[0])

In [ ]:
X_Train = pd.DataFrame(TrainA).copy()
X_Train.drop(LABEL, axis=1, inplace=True)
Y_Train = pd.DataFrame(TrainA[LABEL]).copy()
X_Test = pd.DataFrame(TrainB).copy()
X_Test.drop(LABEL, axis=1, inplace=True)
Y_Test = pd.DataFrame(TrainB[LABEL]).copy()

AmClsArmy = list()
AmClsArmy.append(('LR', LogisticRegression()))
AmClsArmy.append(('ADA', AdaBoostClassifier()))
AmClsArmy.append(('DT', DecisionTreeClassifier()))
AmClsArmy.append(('BG', BaggingClassifier()))
misCls = None
for name, army in AmClsArmy:
    army = army.fit(X_Train, Y_Train)
    predict = army.predict(X_Train)
    tmpCls = pd.DataFrame(TrainA).copy()
    tmpCls['misLabel'] = predict.tolist()
    tmpCls['isMatched'] = np.where(tmpCls[LABEL] == tmpCls['misLabel'], 1, 0)
    tmpCls.drop(tmpCls[tmpCls.isMatched == 1].index, inplace=True)
    misCls = pd.concat([misCls, tmpCls])
    
    predict = army.predict(X_Test)
    tmpCls = pd.DataFrame(TrainB).copy()
    tmpCls['misLabel'] = predict.tolist()
    tmpCls['isMatched'] = np.where(tmpCls[LABEL] == tmpCls['misLabel'], 1, 0)
    tmpCls.drop(tmpCls[tmpCls.isMatched == 1].index, inplace=True)
    misCls = pd.concat([misCls, tmpCls])
    print(name, " : ", len(misCls.axes[0]))

In [ ]:
tmpmisCls = pd.DataFrame(misCls).copy()
tmpmisCls.drop(['misLabel','isMatched'],1, inplace=True)
tmpmisCls = pd.concat([TrainA,tmpmisCls])

X_Train = pd.DataFrame(TrainB).copy()
X_Train.drop(LABEL, axis=1, inplace=True)
Y_Train = pd.DataFrame(TrainB[LABEL]).copy()
X_Test = pd.DataFrame(tmpmisCls).copy()
X_Test.drop(LABEL, axis=1, inplace=True)
Y_Test = pd.DataFrame(tmpmisCls[LABEL]).copy()

BmClsArmy = list()
BmClsArmy.append(('LR', LogisticRegression()))
BmClsArmy.append(('ADA', AdaBoostClassifier()))
BmClsArmy.append(('DT', DecisionTreeClassifier()))
BmClsArmy.append(('BG', BaggingClassifier()))
for name, army in BmClsArmy:
    army = army.fit(X_Train, Y_Train)
    predict = army.predict(X_Train)
    tmpCls = pd.DataFrame(TrainB).copy()
    tmpCls['misLabel'] = predict.tolist()
    tmpCls['isMatched'] = np.where(tmpCls[LABEL] == tmpCls['misLabel'], 1, 0)
    tmpCls.drop(tmpCls[tmpCls.isMatched == 1].index, inplace=True)
    misCls = pd.concat([misCls, tmpCls])
    
    predict = army.predict(X_Test)
    tmpCls = pd.DataFrame(tmpmisCls).copy()
    tmpCls['misLabel'] = predict.tolist()
    tmpCls['isMatched'] = np.where(tmpCls[LABEL] == tmpCls['misLabel'], 1, 0)
    tmpCls.drop(tmpCls[tmpCls.isMatched == 1].index, inplace=True)
    misCls = pd.concat([misCls, tmpCls])
    print(name, " : ", len(misCls.axes[0]))

In [ ]:
tmpmisCls = pd.DataFrame(misCls).copy()
tmpmisCls.drop(['misLabel','isMatched'],1, inplace=True)
tmpmisCls = pd.concat([Train70,tmpmisCls])

X_Train = pd.DataFrame(tmpmisCls).copy()
X_Train.drop(LABEL, axis=1, inplace=True)
Y_Train = pd.DataFrame(tmpmisCls[LABEL]).copy()
X_Test = pd.DataFrame(tmpmisCls).copy()
X_Test.drop(LABEL, axis=1, inplace=True)
Y_Test = pd.DataFrame(tmpmisCls[LABEL]).copy()

BmClsArmy = list()
BmClsArmy.append(('LR', LogisticRegression()))
BmClsArmy.append(('ADA', AdaBoostClassifier()))
BmClsArmy.append(('DT', DecisionTreeClassifier()))
BmClsArmy.append(('BG', BaggingClassifier()))
for name, army in BmClsArmy:
    army = army.fit(X_Train, Y_Train)    
    predict = army.predict(X_Test)
    tmpCls = pd.DataFrame(tmpmisCls).copy()
    tmpCls['misLabel'] = predict.tolist()
    tmpCls['isMatched'] = np.where(tmpCls[LABEL] == tmpCls['misLabel'], 1, 0)
    tmpCls.drop(tmpCls[tmpCls.isMatched == 1].index, inplace=True)
    misCls = pd.concat([misCls, tmpCls])
    print(name, " : ", len(misCls.axes[0]))

In [ ]:
PERFORMANCE = pd.DataFrame(columns=['Model','Accuracy','Precision','Recall','F1Score','Time','TotalFeatures','ReducedFeatures','RunCount'])
newRow= {}
X_Train = pd.DataFrame(Train70).copy()
X_Train.drop(LABEL, axis=1, inplace=True)
Y_Train = pd.DataFrame(Train70[LABEL]).copy()

X_Test = pd.DataFrame(Test30).copy()
X_Test.drop(LABEL, axis=1, inplace=True)
Y_Test = pd.DataFrame(Test30[LABEL]).copy()
start = time.time()
stkBaseArmy = list()
stkBaseArmy.append(('LR', LogisticRegression()))
stkBaseArmy.append(('ADA', AdaBoostClassifier()))
stkBaseArmy.append(('DT', DecisionTreeClassifier()))
stkBaseArmy.append(('BG', BaggingClassifier()))
topArmy = GaussianNB()
model = StackingClassifier(estimators=stkBaseArmy, final_estimator=topArmy, cv=2)
model.fit(X_Train, Y_Train)
predict = model.predict(X_Test)
accuracy = metrics.accuracy_score(Y_Test, predict)
print('Base Stacking Accuracy',accuracy)
precision = metrics.precision_score(Y_Test, predict)
recall = metrics.recall_score(Y_Test, predict)
f1 = metrics.f1_score(Y_Test, predict, zero_division=1)
end = time.time()
timeTaken = int(end - start)
newRow = {'Model':DS+'_baseStacking','Accuracy':accuracy,'Precision':precision,'Recall':recall,'F1Score':f1,'Time':timeTaken,'TotalFeatures':len(MISet),'ReducedFeatures':len(FinalSet),'RunCount':runCount}
PERFORMANCE = PERFORMANCE.append(newRow , ignore_index=True)

In [ ]:
misCls.drop(['misLabel','isMatched'],1, inplace=True)
misCls = pd.concat([Train70,misCls])
misCls = misCls.sample(frac=1).reset_index(drop=True)

X_Train = pd.DataFrame(misCls).copy()
X_Train.drop(LABEL, axis=1, inplace=True)
Y_Train = pd.DataFrame(misCls[LABEL]).copy()

X_Test = pd.DataFrame(Test30).copy()
X_Test.drop(LABEL, axis=1, inplace=True)
Y_Test = pd.DataFrame(Test30[LABEL]).copy()
start = time.time()
mClsStkArmy = list()
mClsStkArmy.append(('LR', LogisticRegression()))
mClsStkArmy.append(('ADA', AdaBoostClassifier()))
mClsStkArmy.append(('DT', DecisionTreeClassifier()))
mClsStkArmy.append(('BG', BaggingClassifier()))
topArmy = GaussianNB()
model = StackingClassifier(estimators=mClsStkArmy, final_estimator=topArmy, cv=2)
model.fit(X_Train, Y_Train)
predict = model.predict(X_Test)
accuracy = metrics.accuracy_score(Y_Test, predict)
print('mClsStacking Accuracy',accuracy)
precision = metrics.precision_score(Y_Test, predict)
recall = metrics.recall_score(Y_Test, predict)
f1 = metrics.f1_score(Y_Test, predict, zero_division=1)
end = time.time()
timeTaken = int(end - start)
newRow = {'Model':DS+'_ClsStacking','Accuracy':accuracy,'Precision':precision,'Recall':recall,'F1Score':f1,'Time':timeTaken,'TotalFeatures':len(MISet),'ReducedFeatures':len(FinalSet)-1,'RunCount':runCount}
PERFORMANCE = PERFORMANCE.append(newRow , ignore_index=True)